# Pre-Process MSSIS Port Visits

Cleaning of the Bermuda Case Study MSSIS port visit history dataset provided by Volpe. Upon inspection of the MSSIS AIS historical dataset it was apparent there were coverage issues for Bermuda and swaths of the Atlantic. Initially it was believed that the team could recieve commercial AIS data which was assumed to include satelite data. However, this was not possible due to contractual issues.

In [ ]:
visits = '../data/raw/volpe-bermuda-mssis-port-visits.txt'
bm_wpi = '../data/raw/volpe-bermuda-mssis-wpi-bm.txt'

In [ ]:
out_path = '../data/interim/'

In [ ]:
import pandas as pd
import requests
import geopandas
import seaconex
import json
import seaconex
import numpy as np

from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file

## volpe-bermuda-mssis-wpi-bm.txt

In [ ]:
# !head $bm_wpi

In [ ]:
# !tail $bm_wpi 

In [ ]:
data = seaconex.piped_txt_to_arr(bm_wpi)

In [ ]:
data

In [ ]:
bm_wpi_df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
bm_wpi_df.rename(
    columns = {
        'World_port_index_number': 'mssis_wpi',
        'Region_index': 'mssis_wpi_region_index',
        'Main_port_name': 'mssis_wpi_main_port_name',
        'Wpi_country_code': 'mssis_wpi_country'
    }, inplace=True
)

In [ ]:
bm_wpi_df.to_json(out_path + 'mssis-wpi-bm-ports_v1.json', orient='records')

In [ ]:
# df.info()

## volpe-bermuda-mssis-port-visits.txt

In [ ]:
# !head $visits

In [ ]:
# !tail $visits

In [ ]:
data = seaconex.piped_txt_to_arr(visits)

In [ ]:
# data[:3]

In [ ]:
df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
# df.head()

In [ ]:
# df.info()

In [ ]:
df['ExitTime'] = pd.to_datetime(df['ExitTime'], unit ='s')
df['EntryTime'] = pd.to_datetime(df['EntryTime'], unit ='s')

In [ ]:
df.drop(columns=['from_unixtime(EntryTime)', 'from_unixtime(ExitTime)'], inplace=True)

In [ ]:
df[['MMSI', 'PortIndex']] = df[['MMSI', 'PortIndex']].astype('uint32')

In [ ]:
# df.info()

In [ ]:
df.rename(
    columns = {
        'MMSI': 'vessel_mmsi', 
        'EntryTime': 'ais_port_entry_time', 
        'ExitTime': 'ais_port_exit_time', 
        'PortIndex': 'mssis_wpi', 
        'main_port_name': 'mssis_wpi_main_port_name', 
        'wpi_country_code': 'mssis_wpi_country'
    }, inplace=True
)

In [ ]:
# df.info()

In [ ]:
df.head()

In [ ]:
df.vessel_mmsi.unique()

In [ ]:
df.ais_port_entry_time.unique().size

In [ ]:
df.ais_port_exit_time.unique().size

In [ ]:
df.mssis_wpi.unique().size

In [ ]:
df.mssis_wpi_main_port_name.unique()

In [ ]:
df.mssis_wpi_main_port_name.unique().size

In [ ]:
df.mssis_wpi_country.unique()

In [ ]:
df.mssis_wpi_country.unique().size

In [ ]:
df.columns.str.contains('wpi')

In [ ]:
df.filter(like='wpi').columns

In [ ]:
df.filter(like='wpi').drop_duplicates().reset_index(drop=True)

In [ ]:
# df.groupby(['mssis_wpi','mssis_wpi_main_port_name', 'mssis_wpi_country']).size().reset_index().rename(columns={0:'ais_count'})

In [ ]:
ports = df.filter(like='wpi').drop_duplicates().reset_index(drop=True).to_json(orient='records')

In [ ]:
ports = df.filter(like='wpi').drop_duplicates().reset_index(drop=True).to_json(out_path + 'mssis-port-visit-ports_v1.json')

In [ ]:
# with open(out_path + 'mssis-port-visit-ports_v1.json', 'w') as json_file:
#     json.dump(ports, json_file)

In [ ]:
# df.groupby(['vessel_mmsi', 'mssis_wpi','mssis_wpi_main_port_name', 'mssis_wpi_country']).count()

In [ ]:
# df.sort_values(['vessel_mmsi'],ascending=False).groupby(['mssis_wpi','mssis_wpi_main_port_name', 'mssis_wpi_country']).head(3)

In [ ]:
df.sort_values(['vessel_mmsi','ais_port_entry_time', 'ais_port_exit_time'], ascending=True).head()

In [ ]:
df

In [ ]:
df['event_duration'] = df.ais_port_exit_time - df.ais_port_entry_time

In [ ]:
df

In [ ]:
# d2 = df[df['vessel_mmsi']==211256150].sort_values(['ais_port_entry_time', 'ais_port_exit_time'], ascending=True)

In [ ]:
# len(df.ais_port_entry_time.tolist())

In [ ]:
gdf = read_file('../data/interim/mssis-ais-records_v2.gpkg')
wgs84 = gdf.crs

gdf['ais_time'] = pd.to_datetime(gdf['ais_time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
gdf

In [ ]:
# df3 = df3[df3['vessel_mmsi']==211256150].sort_values(['ais_time'], ascending=True)

In [ ]:
# df3.head()

In [ ]:
# df.ais_port_entry_time.tolist()

In [ ]:
gdf[gdf['ais_time'].isin(df.ais_port_entry_time.tolist())]

In [ ]:
gdf[gdf['ais_time'].isin(df.ais_port_exit_time.tolist())]

In [ ]:
join_col = np.intersect1d(gdf.columns, df.columns).tolist()

In [ ]:
join_col

In [ ]:
df_in = pd.merge(left=gdf, right=df, how='inner', left_on=['ais_time', 'mssis_wpi', 'vessel_mmsi'], right_on=['ais_port_entry_time', 'mssis_wpi', 'vessel_mmsi'])

In [ ]:
df_in['event'] = 'IN'

In [ ]:
df_out = pd.merge(left=gdf, right=df, how='inner', left_on=['ais_time', 'mssis_wpi', 'vessel_mmsi'], right_on=['ais_port_exit_time', 'mssis_wpi', 'vessel_mmsi'])

In [ ]:
df_out['event'] = 'OUT'

In [ ]:
pd.set_option("max_rows", None)

In [ ]:
result = pd.concat([df_in, df_out])

In [ ]:
# result.sort_values(['ais_time', 'vessel_mmsi'], ascending=True)
# result.groupby(['vessel_mmsi'], sort=False)
grouped = result. sort_values(['vessel_mmsi'], ascending=True) \
            .groupby(['vessel_mmsi'], sort=False) \
            .apply(lambda x: x.sort_values(['ais_time'], ascending=True)) \
            .reset_index(drop=True)

In [ ]:
# print(grouped)
grouped.head()

In [ ]:
grouped.info()

In [ ]:
grouped['event_duration'] = grouped['event_duration'].astype('str')

In [ ]:
grouped.to_file(out_path + 'mssis-grouped-ais-port-visit_v1.geojson', driver='GeoJSON')

In [ ]:
grouped.to_file(out_path + 'mssis-grouped-ais-port-visit_v1.gpkg', driver='GPKG')